# Lab 10 : VirSorter2

## Virsorter2

Virsorter2 (https://microbiomejournal.biomedcentral.com/articles/10.1186/s40168-020-00990-y) is a tool for identifying viruses in contigs and bins. It is available on github - https://github.com/jiarong/VirSorter2  For the directions for installing either VirSorter  Option 1 (bioconda version) or the developmental version (Option 2) in your Unity directory. If you install Option 1 create a new directory "VirSorter2" to put your files in. Run the test.fa file as per the github recommendations to make sure you installation works.  VirSorter takes as an input a fasta formatted nucleotide sequence file. There can be multiple fasta sequences in the same file. 


## Interactive and Batch Jobs on Unity

The Unity documentation provides an overview on running jobs - https://unity.rc.umass.edu/docs/#slurm/ .  Many bioinformatics applications take 10s of minutes to 10s of hours to run so there is no sense in running them interactive to wait for the results (and keep your laptop open). `SBATCH` jobs are typically run using a shell script containing multiple commands.



### Shell Scripts

To create a shell script open a text file, add the text `echo "Hello World!"` and save it as `hello.sh`.  To be able to run it the file permissions must be changed. This is done in the terminal using chmod (for more info see https://cets.seas.upenn.edu/answers/chmod.html).

In [1]:
chmod +x hello.sh

SyntaxError: invalid syntax (<ipython-input-1-d5975ed40863>, line 1)

To run VirSorter using SBATCH copy and past the below text into a text file and save it as `virsorter_sbatch.sh`. This file will activate the conda vs2 environment and run virsorter. Remember to change the permissions.

In [ ]:
#!/bin/bash
#SBATCH -J virsorter_sbatch
#SBATCH -o filename_%j.txt
#SBATCH -e filename_%j.err
#SBATCH -c 4  # Number of Cores per Task
#SBATCH --time=02:00:00
#SBATCH --mem=16G

# With Slurm jobs, your ~/.bashrc is not sourced, 
# so it doesn’t initialize the shell for Conda environments and you will get an error message
# Add this line before activating the environment:
eval "$(conda shell.bash hook)"

# Activate the virsorter vs2 conda environment
conda activate vs2

# Run virsorter
virsorter run -w test_sbatch.out -i test.fa --min-length 1500 -j 4 all

To run the job

In [ ]:
sbatch virsorter_sbatch.sh

More sbatch options can be viewed using `sbatch -h`.  Here is an overview of the more common options for viewing and stopping jobs https://docs.ycrc.yale.edu/clusters-at-yale/job-scheduling/

In [ ]:
# List your queued and running jobs:

squeue --me

# Cancel a queued job or kill a running job, e.g. a job with ID 12345:

scancel 12345

# Check status of a job, e.g. a job with ID 12345:

sacct -j 12345

# Check how efficiently a job ran, e.g. a job with ID 12345:

seff 12345

## Running virsorter on all your bins (or contigs)

Process your sample reads through trimmomatic and metaspades.  Use both the assembled contigs and the reads as inputs in the binning programs when the option is available. For this lab use either `Metabat` and/or `Concoct` to bin your reads as these provide fasta nucleotide files of your bins (and in the case of Metabit the remaining contigs). Download and transfer the binning results which include the fasta sequences from KBase to Unity.  

On Unity make a directory within VirSorter2 with the fasta files. For example `SampleA1_metabat_contigs`

In [ ]:
# virsorter_run.py
# A program to run virsorter on multiple fasta files in the same directory

import os, sys, glob

# move into the directory containing the sequences
os.chdir("SampleA1_metabat_contigs")

# get a list of the fasta files
fasta_files = glob.glob('*.fa')

# run virsorter on each of these files
for file in fasta_files :
    os.system ('virsorter {}'.format('run -w ' + file + '.out -i ' + file + ' --min-length 1500 -j 4 all')) 
    
# Move back to your virsorter directory
os.chdir("..")

We don't want to run the above in interactive mode, so let's make a sbatch file to run it in batch mode. Notice that instead of running virsorter within the script we are calling our python program. Save the script as `vsorter_sbatch_dir.sh` and make it executable

In [ ]:
#!/bin/bash
#SBATCH -J virsorter_test
#SBATCH -o filename_%j.txt
#SBATCH -e filename_%j.err
#SBATCH -c 4  # Number of Cores per Task
#SBATCH --time=02:00:00
#SBATCH --mem=16G

# With Slurm jobs, your ~/.bashrc is not sourced, 
# so it doesn’t initialize the shell for Conda environments and you will get an error message
# Add this line before activating the environment:
eval "$(conda shell.bash hook)"

conda activate vs2

python virsorter_run.py

In [ ]:
# Run the script within the VirSorter2 directory
sbatch vsorter_sbatch_dir.sh

It may take many hours to a day for the job to finish. You can check on the job using the above commands. You may also want to consider allocate more resources to the job. The end results should be a seriers of sub-directories named after the fasta files.

## Exercises
 
Your challenge now is to write a Python script that goes into each sub-directory. Opens each `final-viral-score.tsv` file and adds it to a single summary file that has the name of the original fasta file in the first column followed by the columns in the corresponding `final-viral-score.tsv`.  Submit this file on Moodle by Dec. 1